In [ ]:
# 23-12-20 : NSGA iii proposed method crossover

In [ ]:
import time, array, random, copy, math
from itertools import chain
from operator import attrgetter, itemgetter

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as a3
from matplotlib.path import Path
import matplotlib.patches as patches

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.4 MB/s eta 0:00:00


In [ ]:
from deap import algorithms, base, benchmarks, tools, creator
import numpy as np

In [ ]:
import copy,random
import numpy as np
from deap import tools

# modules

In [ ]:
class ReferencePoint(list):
    '''A reference point exists in objective space an has a set of individuals
    associated to it.'''
    def __init__(self, *args):
        list.__init__(self, *args)
        self.associations_count = 0
        self.associations = []

In [ ]:
def generate_reference_points(num_objs, num_divisions_per_obj):
    '''Generates reference points for NSGA-III selection. This code is based on
    `jMetal NSGA-III implementation <https://github.com/jMetal/jMetal>`_.
    '''
    def gen_refs_recursive(work_point, num_objs, left, total, depth):
        if depth == num_objs - 1:
            work_point[depth] = left/total
            ref = ReferencePoint(copy.deepcopy(work_point))
            return [ref]
        else:
            res = []
            for i in range(left):
                work_point[depth] = i/total
                res = res + gen_refs_recursive(work_point, num_objs, left-i, total, depth+1)
            return res
    return gen_refs_recursive([0]*num_objs, num_objs, num_objs*num_divisions_per_obj,
                              num_objs*num_divisions_per_obj, 0)

In [ ]:
def find_ideal_point(individuals):
    'Finds the ideal point from a set individuals.'
    current_ideal = [np.infty] * len(individuals[0].fitness.values)
    for ind in individuals:
        # Use wvalues to accomodate for maximization and minimization problems.
        current_ideal = np.minimum(current_ideal,
                                  np.multiply(ind.fitness.wvalues, -1))
    return current_ideal

In [ ]:
def find_extreme_points(individuals):
    'Finds the individuals with extreme values for each objective function.'
    return [sorted(individuals, key=lambda ind:ind.fitness.wvalues[o] * -1)[-1]
            for o in range(len(individuals[0].fitness.values))]

In [ ]:
def construct_hyperplane(individuals, extreme_points):
    'Calculates the axis intersects for a set of individuals and its extremes.'
    def has_duplicate_individuals(individuals):
        for i in range(len(individuals)):
            for j in range(i+1, len(individuals)):
                if individuals[i].fitness.values == individuals[j].fitness.values:
                    return True
        return False

    num_objs = len(individuals[0].fitness.values)

    if has_duplicate_individuals(extreme_points):
        intercepts = [extreme_points[m].fitness.values[m] for m in range(num_objs)]
    else:
        b = np.ones(num_objs)
        A = [point.fitness.values for point in extreme_points]
        x = np.linalg.solve(A,b)
        intercepts = 1/x
    return intercepts

In [ ]:
def normalize_objective(individual, m, intercepts, ideal_point, epsilon=1e-20):
    'Normalizes an objective.'
    # Numeric trick present in JMetal implementation.
    if np.abs(intercepts[m]-ideal_point[m] > epsilon):
        return individual.fitness.values[m] / (intercepts[m]-ideal_point[m])
    else:
        return individual.fitness.values[m] / epsilon

In [ ]:
def normalize_objectives(individuals, intercepts, ideal_point):
    '''Normalizes individuals using the hyperplane defined by the intercepts as
    reference. Corresponds to Algorithm 2 of Deb & Jain (2014).'''
    num_objs = len(individuals[0].fitness.values)

    for ind in individuals:
        ind.fitness.normalized_values = list([normalize_objective(ind, m,
                                                                  intercepts, ideal_point)
                                                                  for m in range(num_objs)])
    return individuals

In [ ]:
def perpendicular_distance(direction, point):
    k = np.dot(direction, point) / np.sum(np.power(direction, 2))
    d = np.sum(np.power(np.subtract(np.multiply(direction, [k] * len(direction)), point) , 2))
    return np.sqrt(d)

In [ ]:
def associate(individuals, reference_points):
    '''Associates individuals to reference points and calculates niche number.
    Corresponds to Algorithm 3 of Deb & Jain (2014).'''
    pareto_fronts = tools.sortLogNondominated(individuals, len(individuals))
    num_objs = len(individuals[0].fitness.values)

    for ind in individuals:
        rp_dists = [(rp, perpendicular_distance(ind.fitness.normalized_values, rp))
                    for rp in reference_points]
        best_rp, best_dist = sorted(rp_dists, key=lambda rpd:rpd[1])[0]
        ind.reference_point = best_rp
        ind.ref_point_distance = best_dist
        best_rp.associations_count +=1 # update de niche number
        best_rp.associations += [ind]
    print("pareto front in associate", '\n', pareto_fronts)

In [ ]:
def niching_select(individuals, k):
    '''Secondary niched selection based on reference points. Corresponds to
    steps 13-17 of Algorithm 1 and to Algorithm 4.'''
    if len(individuals) == k:
        return individuals

    #individuals = copy.deepcopy(individuals)

    ideal_point = find_ideal_point(individuals)
    extremes = find_extreme_points(individuals)
    intercepts = construct_hyperplane(individuals, extremes)
    normalize_objectives(individuals, intercepts, ideal_point)

    reference_points = generate_reference_points(len(individuals[0].fitness.values), num_divisions_per_obj)

    associate(individuals, reference_points)

    res = []
    while len(res) < k:
        min_assoc_rp = min(reference_points, key=lambda rp: rp.associations_count)
        min_assoc_rps = [rp for rp in reference_points if rp.associations_count == min_assoc_rp.associations_count]
        chosen_rp = min_assoc_rps[random.randint(0, len(min_assoc_rps)-1)]

        #print('Rps',min_assoc_rp.associations_count, chosen_rp.associations_count, len(min_assoc_rps))

        associated_inds = chosen_rp.associations

        if chosen_rp.associations:
            if chosen_rp.associations_count == 0:
                sel = min(chosen_rp.associations, key=lambda ind: ind.ref_point_distance)
            else:
                sel = chosen_rp.associations[random.randint(0, len(chosen_rp.associations)-1)]
            res += [sel]
            chosen_rp.associations.remove(sel)
            chosen_rp.associations_count += 1
            individuals.remove(sel)
        else:
            reference_points.remove(chosen_rp)
    return res

# environment dev

In [ ]:
number_of_variables = 3 # number of variables
number_of_obj = 4 # number of objectives
num_divisions_per_obj = 4 # no of divisions per object
num_objs = number_of_obj
P = 20 #  no of divisions of the hyperplane
H = 100 # factorial(number_of_obj + P - 1) / (factorial(P) * factorial(number_of_obj - 1))
number_of_gen = 200 # max no of generations
bounds_low, bounds_up = 0, 1
MU = int(H + (number_of_variables - H % number_of_variables)) # no of candidates in population
max_pop = 200
max_gen = 10

In [ ]:
creator.create("FitnessMin3", base.Fitness, weights=(-1.0,) * number_of_obj)
creator.create("Individual3", array.array, typecode='d',
               fitness=creator.FitnessMin3)

In [ ]:
def sel_nsga_iii(individuals, k):
    '''Implements NSGA-III selection as described in
    Deb, K., & Jain, H. (2014). An Evolutionary Many-Objective Optimization
    Algorithm Using Reference-Point-Based Nondominated Sorting Approach,
    Part I: Solving Problems With BoxConstraints. IEEE Transactions on
    Evolutionary Computation, 18(4), 577–601. doi:10.1109/TEVC.2013.2281535.
    '''
    assert len(individuals) >= k

    if len(individuals)==k:
        return individuals

    # Algorithm 1 steps 4--8
    fronts = tools.sortLogNondominated(individuals, len(individuals))
    print("front in main", '\n', fronts)


    limit = 0
    res =[]
    for f, front in enumerate(fronts):
        res += front
        if len(res) > k:
            limit = f
            break
    # Algorithm 1 steps
    selection = []
    if limit > 0:
        for f in range(limit):
            selection += fronts[f]

    # complete selected inividuals using the referece point based approach
    selection += niching_select(fronts[limit], k - len(selection))
    global pareto_front
    pareto_front = selection
    return selection

__all__ = ["sel_nsga_iii"]

In [ ]:
def prepare_toolbox(problem_instance, selection_func, number_of_variables, bounds_low, bounds_up):

    def uniform(low, up, size=None):
        try:
            return [random.uniform(a, b) for a, b in zip(low, up)]
        except TypeError:
            return [random.uniform(a, b) for a, b in zip([low] * size, [up] * size)]

    toolbox = base.Toolbox()

    toolbox.register('evaluate', problem_instance)
    toolbox.register('select', selection_func)

    toolbox.register("attr_float", uniform, bounds_low, bounds_up, number_of_variables)
    toolbox.register("individual", tools.initIterate, creator.Individual3, toolbox.attr_float)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxSimulatedBinaryBounded,
                     low=bounds_low, up=bounds_up, eta=20.0)
    toolbox.register("mutate", tools.mutPolynomialBounded,
                     low=bounds_low, up=bounds_up, eta=20.0,
                     indpb=1.0/number_of_variables)
    toolbox.register('select', selection_func)

    toolbox.pop_size = max_pop   # max population size allowed
    toolbox.max_gen = max_gen    # max number of iteration
    toolbox.mut_prob = 1/number_of_variables
    toolbox.cross_prob = 0.3

    return toolbox

In [ ]:
toolbox = prepare_toolbox(lambda ind: benchmarks.dtlz2(ind, number_of_obj),
                          sel_nsga_iii, number_of_variables,
                          bounds_low, bounds_up)

In [ ]:
# creating a population and evaluation

In [ ]:
pop = toolbox.population(n=max_pop)

for ind in pop:
    ind.fitness.values = toolbox.evaluate(ind)

In [ ]:
fig = plt.figure(figsize=(9,9))
ax = fig.add_subplot(111, projection='3d')

# the coordinate origin (black + sign)
ax.scatter(0,0,0, c='k', marker='+', s=100)

# the population (purple)
for ind in pop:
    ax.scatter(ind.fitness.values[0],
               ind.fitness.values[1],
               ind.fitness.values[2],
               color='purple',
               s=30, marker='o')

# ideal point (red star)
ideal_point = find_ideal_point(pop)
ax.scatter(ideal_point[0], ideal_point[1], ideal_point[2],
           s=50, marker='*', color='salmon')

# extreme points marked (red)
extremes = find_extreme_points(pop)
for i,ex in enumerate(extremes):
    ax.scatter(ex.fitness.values[0],
               ex.fitness.values[1],
               ex.fitness.values[2], s=30, marker='o', color='r')

# intercepts (in green)
intercepts = construct_hyperplane(pop, extremes)
verts = [(intercepts[0], 0, 0), (0, intercepts[1], 0), (0, 0, intercepts[2])]

for vert in verts:
    ax.scatter(vert[0], vert[1], vert[2],  color='forestgreen', s=100, marker='.')

tri = a3.art3d.Poly3DCollection([verts])
tri.set_color('lightgreen')
tri.set_alpha(0.11)
tri.set_edgecolor('lightgreen')
ax.add_collection3d(tri)

# normalized objectives (navy)
normalize_objectives(pop, intercepts, ideal_point)

for ind in pop:
    ax.scatter(ind.fitness.normalized_values[0],
               ind.fitness.normalized_values[1],
               ind.fitness.normalized_values[2], color='navy', marker='o')

# reference points (gray)
rps = generate_reference_points(num_objs, num_divisions_per_obj)
for rp in rps:
    ax.scatter(rp[0], rp[1], rp[2], marker='o', color='gray')

# final figure details
ax.set_xlabel('$f_1()$', fontsize=15)
ax.set_ylabel('$f_2()$', fontsize=15)
ax.set_zlabel('$f_3()$', fontsize=15)
ax.view_init(elev=9, azim=-40)
plt.autoscale(tight=True)

In [ ]:
def nsga_iii(toolbox, stats = None, verbose=False): # stats=None
    population = toolbox.population(n=toolbox.pop_size)
    return algorithms.eaMuPlusLambda(population, toolbox,
                              mu=toolbox.pop_size,
                              lambda_=toolbox.pop_size,
                              cxpb=toolbox.cross_prob,
                              mutpb=toolbox.mut_prob,
                              ngen=toolbox.max_gen,
                              stats=stats, verbose=verbose)

# implement

In [ ]:
%time res, logbook = nsga_iii(toolbox)

front in main 
 [[Individual3('d', [0.788553373361326, 0.873309904602707, 0.9988298742697843]), Individual3('d', [0.998631419766675, 0.5468868026549907, 0.8905191653330159]), Individual3('d', [0.9153976596660394, 0.9935688286650103, 0.03134146004724747]), Individual3('d', [0.9579965666141977, 0.9647765622887187, 0.7604401844299161]), Individual3('d', [0.9528466160808524, 0.9078955297475918, 0.9173066999406385]), Individual3('d', [0.9528466160808524, 0.9078744767888939, 0.9169083751567394]), Individual3('d', [0.9979929934193933, 0.04516573891915254, 0.6768836122682167]), Individual3('d', [0.612187172144661, 0.9876184287669155, 0.8287957179190963]), Individual3('d', [0.9683466348526814, 0.9360154377531529, 0.5043855124415964]), Individual3('d', [0.8237005954251414, 0.26237461458572997, 0.9900369687834365]), Individual3('d', [0.46659694161559406, 0.8717356843018619, 0.982571269029185]), Individual3('d', [0.5035253764229559, 0.2382837766036724, 0.9954303615754608]), Individual3('d', [0.503

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

for ind in res:
    ax.scatter(ind.fitness.values[0],
               ind.fitness.values[1],
               ind.fitness.values[2], marker='o', color='mediumpurple')

ax.set_xlabel('$f_1()$', fontsize=15)
ax.set_ylabel('$f_2()$', fontsize=15)
ax.set_zlabel('$f_3()$', fontsize=15)
ax.view_init(elev=11, azim=-21)
plt.autoscale(tight=True)

In [ ]:
optimal_values = tools.selBest(res, 10)
optimal_values

[Individual3('d', [0.05575189383320616, 0.5873613605430155, 0.9953644261987832]),
 Individual3('d', [0.054818269940734576, 0.9951298208333199, 0.3747824276619899]),
 Individual3('d', [0.8302352798999069, 0.9589524311666227, 0.7311287480751595]),
 Individual3('d', [0.8285787611250433, 0.9589524311666227, 0.7172354506873327]),
 Individual3('d', [0.8103067852854122, 0.6526089527644467, 0.9614121956281373]),
 Individual3('d', [0.8103067852854122, 0.6526089527644467, 0.9614121956281373]),
 Individual3('d', [0.06109124889748514, 0.8137838496476895, 0.9614121956281373]),
 Individual3('d', [0.06917103645675904, 0.6447610828704659, 0.9747808692659444]),
 Individual3('d', [0.878312413232578, 0.7792403480724017, 0.7884126341853941]),
 Individual3('d', [0.12205419944467771, 0.8160321359575173, 0.9478278201209833])]

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

for ind in optimal_values:
    ax.scatter(ind.fitness.values[0],
               ind.fitness.values[1],
               ind.fitness.values[2], marker='o', color='mediumpurple')

ax.set_xlabel('$f_1()$', fontsize=15)
ax.set_ylabel('$f_2()$', fontsize=15)
ax.set_zlabel('$f_3()$', fontsize=15)
ax.view_init(elev=11, azim=-21)
plt.autoscale(tight=True)

In [ ]:
fig = plt.figure(figsize=(9,9))
ax = fig.add_subplot(111, projection='3d')

# the coordinate origin (black + sign)
ax.scatter(0,0,0, c='k', marker='+', s=100)

# the population (purple)
for ind in res:
    ax.scatter(ind.fitness.values[0],
               ind.fitness.values[1],
               ind.fitness.values[2],
               color='purple',
               s=30, marker='o')

# ideal point (red star)
ideal_point = find_ideal_point(res)
ax.scatter(ideal_point[0], ideal_point[1], ideal_point[2],
           s=50, marker='*', color='salmon')

# extreme points marked (red)
extremes = find_extreme_points(res)
for i,ex in enumerate(extremes):
    ax.scatter(ex.fitness.values[0],
               ex.fitness.values[1],
               ex.fitness.values[2], s=30, marker='o', color='r')

# intercepts (in green)
intercepts = construct_hyperplane(res, extremes)
verts = [(intercepts[0], 0, 0), (0, intercepts[1], 0), (0, 0, intercepts[2])]

for vert in verts:
    ax.scatter(vert[0], vert[1], vert[2],  color='forestgreen', s=100, marker='.')

tri = a3.art3d.Poly3DCollection([verts])
tri.set_color('lightgreen')
tri.set_alpha(0.11)
tri.set_edgecolor('lightgreen')
ax.add_collection3d(tri)

# normalized objectives (navy)
normalize_objectives(res, intercepts, ideal_point)

for ind in pop:
    ax.scatter(ind.fitness.normalized_values[0],
               ind.fitness.normalized_values[1],
               ind.fitness.normalized_values[2], color='navy', marker='o')

# reference points (gray)
rps = generate_reference_points(num_objs, num_divisions_per_obj)
for rp in rps:
    ax.scatter(rp[0], rp[1], rp[2], marker='o', color='gray')

# final figure details
ax.set_xlabel('$f_1()$', fontsize=15)
ax.set_ylabel('$f_2()$', fontsize=15)
ax.set_zlabel('$f_3()$', fontsize=15)
ax.view_init(elev=9, azim=-40)
plt.autoscale(tight=True)

In [ ]:
fig = plt.figure(figsize=(9,9))
ax = fig.add_subplot(111, projection='3d')

# the coordinate origin (black + sign)
ax.scatter(0,0,0, c='k', marker='+', s=100)

# the population (purple)
for ind in optimal_values:
    ax.scatter(ind.fitness.values[0],
               ind.fitness.values[1],
               ind.fitness.values[2],
               color='purple',
               s=30, marker='o')

# ideal point (red star)
ideal_point = find_ideal_point(optimal_values)
ax.scatter(ideal_point[0], ideal_point[1], ideal_point[2],
           s=50, marker='*', color='salmon')

# extreme points marked (red)
extremes = find_extreme_points(optimal_values)
for i,ex in enumerate(extremes):
    ax.scatter(ex.fitness.values[0],
               ex.fitness.values[1],
               ex.fitness.values[2], s=30, marker='o', color='r')

# intercepts (in green)
intercepts = construct_hyperplane(optimal_values, extremes)
verts = [(intercepts[0], 0, 0), (0, intercepts[1], 0), (0, 0, intercepts[2])]

for vert in verts:
    ax.scatter(vert[0], vert[1], vert[2],  color='forestgreen', s=100, marker='.')

tri = a3.art3d.Poly3DCollection([verts])
tri.set_color('lightgreen')
tri.set_alpha(0.11)
tri.set_edgecolor('lightgreen')
ax.add_collection3d(tri)

# normalized objectives (navy)
normalize_objectives(optimal_values, intercepts, ideal_point)

for ind in pop:
    ax.scatter(ind.fitness.normalized_values[0],
               ind.fitness.normalized_values[1],
               ind.fitness.normalized_values[2], color='navy', marker='o')

# reference points (gray)
rps = generate_reference_points(num_objs, num_divisions_per_obj)
for rp in rps:
    ax.scatter(rp[0], rp[1], rp[2], marker='o', color='gray')

# final figure details
ax.set_xlabel('$f_1()$', fontsize=15)
ax.set_ylabel('$f_2()$', fontsize=15)
ax.set_zlabel('$f_3()$', fontsize=15)
ax.view_init(elev=9, azim=-40)
plt.autoscale(tight=True)

In [ ]:
# reverse mapping

In [ ]:
# three variables :
# Promo budget(5000-15000),
# Stock unit (2000-12000),
# margin (30%-70%)

In [ ]:
optimal_values

[Individual3('d', [0.05575189383320616, 0.5873613605430155, 0.9953644261987832]),
 Individual3('d', [0.054818269940734576, 0.9951298208333199, 0.3747824276619899]),
 Individual3('d', [0.8302352798999069, 0.9589524311666227, 0.7311287480751595]),
 Individual3('d', [0.8285787611250433, 0.9589524311666227, 0.7172354506873327]),
 Individual3('d', [0.8103067852854122, 0.6526089527644467, 0.9614121956281373]),
 Individual3('d', [0.8103067852854122, 0.6526089527644467, 0.9614121956281373]),
 Individual3('d', [0.06109124889748514, 0.8137838496476895, 0.9614121956281373]),
 Individual3('d', [0.06917103645675904, 0.6447610828704659, 0.9747808692659444]),
 Individual3('d', [0.878312413232578, 0.7792403480724017, 0.7884126341853941]),
 Individual3('d', [0.12205419944467771, 0.8160321359575173, 0.9478278201209833])]

In [ ]:
max_pb, min_pb = 15000, 5000
max_su, min_su = 12000, 2000
max_ma, min_ma = 70, 30

In [ ]:
optimal_sols = []
for i in optimal_values:
  pb = i[0] * (max_pb-min_pb) + min_pb
  su = i[1] * (max_su-min_su) + min_su
  ma = i[2] * (max_ma-min_ma) + min_ma
  optimal_sols.append([pb,su,ma])
optimal_sols

[[5557.518938332061, 7873.613605430155, 69.81457704795133],
 [5548.182699407345, 11951.298208333199, 44.99129710647959],
 [13302.352798999069, 11589.524311666226, 59.24514992300638],
 [13285.787611250433, 11589.524311666226, 58.68941802749331],
 [13103.067852854121, 8526.089527644468, 68.45648782512549],
 [13103.067852854121, 8526.089527644468, 68.45648782512549],
 [5610.912488974851, 10137.838496476896, 68.45648782512549],
 [5691.710364567591, 8447.610828704659, 68.99123477063777],
 [13783.12413232578, 9792.403480724017, 61.536505367415764],
 [6220.541994446778, 10160.321359575173, 67.91311280483933]]

In [ ]:
# 4 objectives
# Inventory clearance : minimum stock unit
# maximise revenue : maximum margin
# maximise lift : maximum margin
# optimise budget : minimum promo budget

In [ ]:
def column(matrix, i):
    return [row[i] for row in matrix]

In [ ]:
# Inventory clearance : minimum stock unit(1)
t = [row[1] for row in optimal_sols]
opt_inv_cls = optimal_sols[t.index(min(t))]
# maximise revenue : maximum margin(2)
t = [row[2] for row in optimal_sols]
opt_max_rev = optimal_sols[t.index(max(t))]
# maximise lift : maximum margin(3)
t = [row[2] for row in optimal_sols]
opt_max_lif = optimal_sols[t.index(max(t))]
# optimise budget : minimum promo budget
t = [row[0] for row in optimal_sols]
opt_prm_bgt = optimal_sols[t.index(min(t))]

In [ ]:
print("inventory clearance", "\npromo budget", round(opt_inv_cls[0],2), "\nstock unit", round(opt_inv_cls[1],2), "\nmargin", round(opt_inv_cls[2],2),"\n--------------------\n")
print("maximise revenue", "\npromo budget", round(opt_max_rev[0],2), "\nstock unit", round(opt_max_rev[1],2), "\nmargin", round(opt_max_rev[2],2),"\n--------------------\n")
print("maximise lift", "\npromo budget", round(opt_max_lif[0],2), "\nstock unit", round(opt_max_lif[1],2), "\nmargin", round(opt_max_lif[2],2),"\n--------------------\n")
print("optimise budget", "\npromo budget", round(opt_prm_bgt[0],2), "\nstock unit", round(opt_prm_bgt[1],2), "\nmargin", round(opt_prm_bgt[2],2),"\n--------------------\n")

inventory clearance 
promo budget 5557.52 
stock unit 7873.61 
margin 69.81 
--------------------

maximise revenue 
promo budget 5557.52 
stock unit 7873.61 
margin 69.81 
--------------------

maximise lift 
promo budget 5557.52 
stock unit 7873.61 
margin 69.81 
--------------------

optimise budget 
promo budget 5548.18 
stock unit 11951.3 
margin 44.99 
--------------------



In [ ]:
np.round(np.array(optimal_sols),2)

In [ ]:
for ind in res:
  print(ind)
  print(ind.fitness.values)

In [ ]:
num_objs*num_divisions_per_obj

16